# $\text{Deutsch-Jozsa Algorithm}$

In [11]:
from qiskit.quantum_info import partial_trace
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np

### $\text{Deutsch-Jozsa Procedure}$

In [12]:
def deutsch_jozsa(oracle, n):
    qc = QuantumCircuit(n + 1, n)

    # Initialize the ancilla qubit
    qc.x(n)
    qc.h(n)

    # H-gate for every input qubits
    qc.h(range(n))

    # Apply any oracle
    qc.append(oracle, range(n + 1))

    # H-gate again for every input qubits
    qc.h(range(n))

    # Measure all qubits (except ancilla qubits)
    qc.measure(np.arange(n), np.arange(n))

    return qc

### $\text{Prepare the Test Oracle}$

In [13]:
# Example for constant oracle (identity)
def constant_oracle(n):
    qc = QuantumCircuit(n + 1, name="Constant")
    qc.to_gate()
    return qc

# Example for balanced oracle (C-NOT)
def balanced_oracle(n):
    qc = QuantumCircuit(n + 1, name="Balanced")
    for i in range(n):
        qc.cx(i, n)
    qc.to_gate()
    return qc

### $\text{Apply the Oracle to Algorithm}$

In [16]:
# Initialization
n = 3                       # number of qubits
oracle = constant_oracle(n) # the oracle

# Apply the Deutsch-Jozsa Algorithm
dj_circ = deutsch_jozsa(oracle, n)

# Show the circuit
dj_circ.draw()

┌───┐     ┌───────────┐┌───┐┌─┐      
q_0: ┤ H ├─────┤0          ├┤ H ├┤M├──────
     ├───┤     │           │├───┤└╥┘┌─┐   
q_1: ┤ H ├─────┤1          ├┤ H ├─╫─┤M├───
     ├───┤     │  Constant │├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├─────┤2          ├┤ H ├─╫──╫─┤M├
     ├───┤┌───┐│           │└───┘ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├┤3          ├──────╫──╫──╫─
     └───┘└───┘└───────────┘      ║  ║  ║ 
c: 3/═════════════════════════════╩══╩══╩═
                                  0  1  2

### $\text{Simulation}$

In [17]:
simulator = Aer.get_backend("statevector_simulator")
job = transpile(dj_circ, simulator)
result = simulator.run(job).result()

# Take the amplitude for every state
statevector = result.get_statevector()

# Exclude the ancilla qubits
reduced_state = partial_trace(statevector, [3])

# Directly takes the amplitude
amplitudes = reduced_state.probabilities_dict()
print("Amplitudo probabilitas tanpa qubit ancilla: ")
for state, prob in amplitudes.items():
    print(f"|{state}> : {prob:.4f}")

Amplitudo probabilitas tanpa qubit ancilla: 
|000> : 1.0000
